In [31]:
import numpy as np
import random
from random import randint
from gurobipy import *
import pandas as pd
from random import seed

In [32]:
Manufacturing_plants = 2
Distribution = 3
Market = 4
Products = 2
Outsourced = 2
dem_recovery = 0.8
p = 0.2
epsilon = 25

In [33]:
# Scenario parameters
a_si = [[1,1], [1,0], [0,1]] # don't include [0,0]
b_sj = [[1,1,1], [1,0,1], [1,1,0], [1,0,0], [0,1,1], [0,1,0], [0,0,1]] # don't include [0,0,0]

In [34]:
Scenarios = [[x,y] for x in a_si for y in b_sj]

In [35]:
Scenarios

[[[1, 1], [1, 1, 1]],
 [[1, 1], [1, 0, 1]],
 [[1, 1], [1, 1, 0]],
 [[1, 1], [1, 0, 0]],
 [[1, 1], [0, 1, 1]],
 [[1, 1], [0, 1, 0]],
 [[1, 1], [0, 0, 1]],
 [[1, 0], [1, 1, 1]],
 [[1, 0], [1, 0, 1]],
 [[1, 0], [1, 1, 0]],
 [[1, 0], [1, 0, 0]],
 [[1, 0], [0, 1, 1]],
 [[1, 0], [0, 1, 0]],
 [[1, 0], [0, 0, 1]],
 [[0, 1], [1, 1, 1]],
 [[0, 1], [1, 0, 1]],
 [[0, 1], [1, 1, 0]],
 [[0, 1], [1, 0, 0]],
 [[0, 1], [0, 1, 1]],
 [[0, 1], [0, 1, 0]],
 [[0, 1], [0, 0, 1]]]

In [36]:
num_Scenarios = len(Scenarios)
p_scen = 1/num_Scenarios

In [37]:
# Product Demand
np.random.seed(0)
demand = np.random.randint(0,50,(num_Scenarios, Products,Market))

In [38]:
# Cost of opening
f_i = [200, 50]
f_j = [75, 100, 50]

In [39]:
# Unit cost of manufacturing product
np.random.seed(0)
Manufacturing_costs = np.random.uniform(0,2, (Manufacturing_plants,Products))

In [40]:
# Unit cost of transporting m from plant to DC
np.random.seed(0)
Transportation_i_j = np.random.uniform(0,2, (Products, Manufacturing_plants, Distribution))

In [41]:
# Unit cost of transporting m from DC to Market Zone
np.random.seed(0)
Transportation_j_k = np.random.uniform(0,2, (Products, Distribution, Market))

In [42]:
# Plant Capacities: Bigger capacities for the more expensive ones
np.random.seed(0)
Capacities_i = np.zeros(Manufacturing_plants) # in volume (metres cubed)
Capacities_i[0] = np.random.randint(800,1000)
Capacities_i[1] = np.random.randint(200,400) 
Capacities_j = np.zeros(Distribution) # in volume (metres cubed)
Capacities_j[0] = np.random.randint(400, 600)
Capacities_j[1] = np.random.randint(600, 800)
Capacities_j[2] = np.random.randint(200,400)
Capacities_l = np.random.randint(50,100, (Products,Outsourced)) # in terms of products 

In [43]:
# Cost of purchasing product m from supplier l (assume only 1 product type from each outsourcer)
np.random.seed(0)
levels = 2
Supplier_cost = np.zeros((levels, Products, Outsourced))
Supplier_cost[0] = np.random.uniform(10,15, (Products, Outsourced))
Supplier_cost[1] = np.random.randint(15,20, (Products, Outsourced))

In [44]:
Supplier_cost

array([[[12.74406752, 13.57594683],
        [13.01381688, 12.72441591]],

       [[16.        , 18.        ],
        [17.        , 19.        ]]])

In [45]:
# Cost of transporting product m from outsourced facility l to j
np.random.seed(0)
T_O_DC = np.random.uniform(2, 5, (Products, Outsourced, Distribution))

In [46]:
# Cost of shipping product m from outsourced facility l to k
np.random.seed(0)
T_O_MZ = np.random.uniform(5, 7,(Products, Outsourced, Market))

In [47]:
# Product volume 
np.random.seed(0)
volume = np.random.uniform(2,3,(Products))

In [48]:
# unit cost of lost sales 
np.random.seed(0)
lost_sales = np.random.randint(20, 30,(Market,Products))

In [49]:
# Initialize model variables
x_i = {} # opening manufacturing plant
x_j = {} # opening DC
U_km = {} # quantity lost sales
V_lm = {} # quantity products purchased from outsourcing
Q_im = {} # quantity produced
Y_ijm = {} # shipping i -> j
Z_jkm = {} # shipping j -> k
T_ljm = {} # shipping l -> j
T_lkm = {} # shipping l -> k
y_lm = {} # indicator variable for step function 

# Dictionaries for analysis 
Cost_dict = {}
Summary_dict = {}
#recovered_demand = np.zeros((num_Scenarios, Products,Market))

## Deterministic Model

In [50]:
grbModel_det = Model('deterministic') # to obtain obj val of each scenario

In [51]:
def SetGurobiModel(scen):
    
    for i in range(Manufacturing_plants):
        x_i[i] = grbModel_det.addVar(vtype = GRB.BINARY)
    
    for j in range(Distribution):
        x_j[j] = grbModel_det.addVar(vtype = GRB.BINARY)
    
    for k in range(Market):
        for m in range(Products):
            U_km[k,m] = grbModel_det.addVar(vtype = GRB.INTEGER)
        
    for m in range(Products):
        for l in range(Outsourced):
            V_lm[m,l] = grbModel_det.addVar(vtype = GRB.INTEGER)
    
    for m in range(Products):
        for i in range(Manufacturing_plants):
            Q_im[m,i] = grbModel_det.addVar(vtype = GRB.INTEGER)
            
    for m in range(Products):
        for i in range(Manufacturing_plants):
            for j in range(Distribution):
                Y_ijm[m,i,j] = grbModel_det.addVar(vtype = GRB.INTEGER)                
    
    for m in range(Products):
        for j in range(Distribution):
            for k in range(Market): 
                Z_jkm[m,j,k] = grbModel_det.addVar(vtype = GRB.INTEGER)                
    
    for m in range(Products):
        for l in range(Outsourced):
            for j in range(Distribution):
                T_ljm[m,l,j] = grbModel_det.addVar(vtype = GRB.INTEGER)      
    
    for m in range(Products):
        for l in range(Outsourced):
            for k in range(Market):
                T_lkm[m,l,k] = grbModel_det.addVar(vtype = GRB.INTEGER)                
    
    for m in range(Products):
        for l in range(Outsourced):
            y_lm[m,l] = grbModel_det.addVar(vtype = GRB.BINARY)
                
    SetGrb_Obj()
    ModelCons(scen)


In [52]:
def SolveModel(scen):
    
    grbModel_det.params.OutputFlag = 0
    grbModel_det.optimize()
    
    # get variable values 
    v_val_x_i = grbModel_det.getAttr('x', x_i)
    v_val_x_j = grbModel_det.getAttr('x', x_j)
    v_val_U_km = grbModel_det.getAttr('x', U_km)
    v_val_V_lm = grbModel_det.getAttr('x', V_lm)
    v_val_Q_im = grbModel_det.getAttr('x', Q_im)
    v_val_Y_ijm = grbModel_det.getAttr('x', Y_ijm)
    v_val_Z_jkm = grbModel_det.getAttr('x', Z_jkm)
    v_val_T_ljm = grbModel_det.getAttr('x', T_ljm)
    v_val_T_lkm = grbModel_det.getAttr('x', T_lkm)
    v_val_y_lm = grbModel_det.getAttr('x', y_lm)
    
    #for m in range(Products):
     #   for k in range(Market):
      #      recovered_demand[scen][m][k] = demand[scen][m][k] - v_val_U_km[k,m]
    obj = grbModel_det.getObjective()
        
    return


In [53]:
# Objective

def SetGrb_Obj():

    grb_expr = LinExpr()

    # Cost of opening
    OC_1 = 0
    OC_2 = 0
    for i in range(Manufacturing_plants):
        OC_1 += f_i[i]*x_i[i]
    for j in range(Distribution):
        OC_2 += f_j[j]*x_j[j]    
        
    # Shipment 
    ship_1 = 0
    ship_2 = 0
    ship_3 = 0
    ship_4 = 0
    
    for i in range(Manufacturing_plants):
        for j in range(Distribution):
            for m in range(Products):
                ship_1 += Transportation_i_j[m][i][j]*Y_ijm[m,i,j]

    for j in range(Distribution):
        for k in range(Market):
            for m in range(Products):
                ship_2 += Transportation_j_k[m][j][k]*Z_jkm[m,j,k]

    for l in range(Outsourced):
        for j in range(Distribution):
            for m in range(Products):
                ship_3 += T_O_DC[m][l][j]*T_ljm[m,l,j]

    for l in range(Outsourced):
        for k in range(Market):
            for m in range(Products):
                ship_4 += T_O_MZ[m][l][k]*T_lkm[m,l,k]
                    
    total_shipment = ship_1 + ship_2 + ship_3 + ship_4

    # Production
    pr_cost = 0
    for i in range(Manufacturing_plants):
        for m in range(Products):
            pr_cost += Manufacturing_costs[i][m]*Q_im[m,i]                

    # Buying from outsource cost
    b_cost = 0
    for l in range(Outsourced):
        for m in range(Products):
            b_cost += Supplier_cost[0][m][l]*V_lm[m,l]*(1 - y_lm[m,l]) + (Supplier_cost[0][m][l]*epsilon + Supplier_cost[1][m][l]*(V_lm[m,l] - epsilon))*y_lm[m,l]

    #Lost Sales
    l_cost = 0
    for k in range(Market):
        for m in range(Products):
            l_cost += lost_sales[k][m]*U_km[k,m]                

    grb_expr += OC_1 + OC_2 + total_shipment + pr_cost + b_cost + l_cost
    
    grbModel_det.setObjective(grb_expr, GRB.MINIMIZE)
    
    return 

In [54]:
# Model Constraints

def ModelCons(scen):
    
    # Network Flow

    grbModel_det.addConstrs(Q_im[m,i] >= quicksum(Y_ijm[m,i,j] for j in range(Distribution)) 
                         for i in range(Manufacturing_plants) for m in range(Products))

    grbModel_det.addConstrs((quicksum(Y_ijm[m,i,j] for i in range(Manufacturing_plants)) +
                         quicksum(T_ljm[m,l,j] for l in range(Outsourced))) >= quicksum(Z_jkm[m,j,k] for k in range(Market))                        
                        for j in range(Distribution) for m in range(Products))

    grbModel_det.addConstrs((quicksum(Z_jkm[m,j,k] for j in range(Distribution)) +
                         quicksum(T_lkm[m,l,k] for l in range(Outsourced)) + U_km[k,m]) >= demand[scen][m][k]
                         for k in range(Market) for m in range(Products))  
        
                    
    # Purchasing Constraints (everything purchased from outsourced facilities must be shipped)
    grbModel_det.addConstrs(V_lm[m,l] >= quicksum(T_ljm[m,l,j] for j in range(Distribution)) + 
                        quicksum(T_lkm[m,l,k] for k in range(Market))  
                        for m in range(Products) for l in range(Outsourced))
    
    
    # Capacity Constraints
    grbModel_det.addConstrs(quicksum(volume[m]*Q_im[m,i] for m in range(Products)) <= Scenarios[scen][0][i]*Capacities_i[i]*x_i[i] 
                         for i in range(Manufacturing_plants))
    
    grbModel_det.addConstrs(quicksum(volume[m]*Y_ijm[m,i,j] for i in range(Manufacturing_plants) for m in range(Products)) +
                        quicksum(volume[m]*T_ljm[m,l,j] for l in range(Outsourced) for m in range(Products)) <= 
                        Scenarios[scen][1][j]*Capacities_j[j]*x_j[j]  
                        for j in range(Distribution))
    
    grbModel_det.addConstrs((V_lm[m,l] <= (Capacities_l[m][l])) 
                        for l in range(Outsourced) for m in range(Products))
    
    
    # Indicator variable constraints for step function (25 is arbitrary)
    grbModel_det.addConstrs(V_lm[m,l] >= (epsilon + 1)*y_lm[m,l] for m in range(Products) for l in range(Outsourced))
    grbModel_det.addConstrs((V_lm[m,l] - (Capacities_l[m][l] - epsilon)*y_lm[m,l]) <= epsilon 
                               for m in range(Products) for l in range(Outsourced))
    
    return

In [55]:
def run_Model(scen):    
        
    SetGurobiModel(scen)
    SolveModel(scen)

In [56]:
scen_objs = []

In [57]:
for s in range(num_Scenarios):
    x_i = {} # opening manufacturing plant
    x_j = {} # opening DC
    U_km = {} # quantity lost sales
    V_lm = {} # quantity products purchased from outsourcing
    Q_im = {} # quantity produced
    Y_ijm = {} # shipping i -> j
    Z_jkm = {} # shipping j -> k
    T_ljm = {} # shipping l -> j
    T_lkm = {} # shipping l -> k
    y_lm = {} # indicator variable for step function 
    run_Model(s)
    scen_objs += [np.round(grbModel_det.objval,2)]

In [58]:
scen_objs

[841.73,
 786.81,
 1172.65,
 672.24,
 1044.35,
 992.28,
 1784.7,
 922.76,
 962.97,
 910.75,
 660.29,
 1100.47,
 1181.99,
 3226.4,
 2900.46,
 1432.73,
 2202.16,
 2406.84,
 1736.26,
 2048.63,
 3096.1]

In [59]:
penalty_weights = [scen_objs[0]/scen_objs[i] for i in range(num_Scenarios)]

In [60]:
penalty_weights

[1.0,
 1.069800841372123,
 0.7178015605679443,
 1.2521272164703083,
 0.805984583712357,
 0.8482787116539687,
 0.47163668963971533,
 0.9121873509905067,
 0.8740978431311464,
 0.9242163052429316,
 1.2747883505732331,
 0.76488227757231,
 0.7121295442431831,
 0.26088829655343415,
 0.29020569151100173,
 0.5875007852142413,
 0.3822292658117485,
 0.3497241195924947,
 0.4847949039890339,
 0.4108745844784075,
 0.27186783372630086]

## Stochastic Model

In [61]:
# Initialize model variables
x_i = {} # opening manufacturing plant
x_j = {} # opening DC
U_km = {} # quantity lost sales
V_lm = {} # quantity products purchased from outsourcing
Q_im = {} # quantity produced
Y_ijm = {} # shipping i -> j
Z_jkm = {} # shipping j -> k
T_ljm = {} # shipping l -> j
T_lkm = {} # shipping l -> k
y_lm = {} # indicator variable for step function 
delta_s = {} # penalty for p-robust
alpha_kms = {} # penalty for demand recovery 

# Dictionaries for analysis 
Cost_dict = {}
Summary_dict = {} 

In [62]:
grbModel = Model('stochastic')

In [63]:
def SetGurobiModel():
    
    for i in range(Manufacturing_plants):
        x_i[i] = grbModel.addVar(vtype = GRB.BINARY)
    
    for j in range(Distribution):
        x_j[j] = grbModel.addVar(vtype = GRB.BINARY)
    
    for s in range(num_Scenarios):
        for k in range(Market):
            for m in range(Products):
                U_km[s,k,m] = grbModel.addVar(vtype = GRB.INTEGER)
        
    for s in range(num_Scenarios):
        for m in range(Products):
            for l in range(Outsourced):
                V_lm[s,m,l] = grbModel.addVar(vtype = GRB.INTEGER)
    
    for s in range(num_Scenarios):
        for m in range(Products):
            for i in range(Manufacturing_plants):
                Q_im[s,m,i] = grbModel.addVar(vtype = GRB.INTEGER)
            
    for s in range(num_Scenarios):  
        for m in range(Products):
            for i in range(Manufacturing_plants):
                for j in range(Distribution):
                    Y_ijm[s,m,i,j] = grbModel.addVar(vtype = GRB.INTEGER)                
    
    for s in range(num_Scenarios):
        for m in range(Products):
            for j in range(Distribution):
                for k in range(Market): 
                    Z_jkm[s,m,j,k] = grbModel.addVar(vtype = GRB.INTEGER)                
    
    for s in range(num_Scenarios):
        for m in range(Products):
            for l in range(Outsourced):
                for j in range(Distribution):
                    T_ljm[s,m,l,j] = grbModel.addVar(vtype = GRB.INTEGER)      
    
    for s in range(num_Scenarios):
        for m in range(Products):
            for l in range(Outsourced):
                for k in range(Market):
                    T_lkm[s,m,l,k] = grbModel.addVar(vtype = GRB.INTEGER)                
    
    for s in range(num_Scenarios):
        for m in range(Products):
            for l in range(Outsourced):
                y_lm[s,m,l] = grbModel.addVar(vtype = GRB.BINARY)
                
    for s in range(num_Scenarios):
        delta_s[s] = grbModel.addVar(vtype = GRB.CONTINUOUS)
    
    for s in range(num_Scenarios):
        for k in range(Market):
            for m in range(Products):
                alpha_kms[s,k,m] = grbModel.addVar(vtype = GRB.CONTINUOUS)
        
                
    SetGrb_Obj()
    ModelCons()

In [64]:
def SolveModel():
    
    grbModel.params.OutputFlag = 0
    grbModel.optimize()
    
    # get variable values 
    v_val_x_i = grbModel.getAttr('x', x_i)
    v_val_x_j = grbModel.getAttr('x', x_j)
    v_val_U_km = grbModel.getAttr('x', U_km)
    v_val_V_lm = grbModel.getAttr('x', V_lm)
    v_val_Q_im = grbModel.getAttr('x', Q_im)
    v_val_Y_ijm = grbModel.getAttr('x', Y_ijm)
    v_val_Z_jkm = grbModel.getAttr('x', Z_jkm)
    v_val_T_ljm = grbModel.getAttr('x', T_ljm)
    v_val_T_lkm = grbModel.getAttr('x', T_lkm)
    v_val_y_lm = grbModel.getAttr('x', y_lm)
    v_val_delta_s = grbModel.getAttr('x', delta_s)
    v_val_alpha_kms = grbModel.getAttr('x', alpha_kms)
    
    obj = grbModel.getObjective()
    print("obj val: ", obj.getValue())   
    
    Summary_dict['ObjVal'] = grbModel.objval
    Summary_dict["OpenMPs"] = np.sum(v_val_x_i.values())
    Summary_dict["OpenDCs"] = np.sum(v_val_x_j.values())
    Cost_dict["Opening"] =  get_opening_costs(v_val_x_i, v_val_x_j)
    Cost_dict["r_Penalties"] = get_r_penalty_costs(v_val_delta_s)
    Cost_dict["d_Penalties"] = get_d_penalty_costs(v_val_alpha_kms)
    
    for s in range(num_Scenarios):        
        Summary_dict["Purchasing_" + str(s)] = sum([v_val_V_lm[(s,m,l)] for m in range(Products) for l in range(Outsourced)])    
        Summary_dict["Production_" + str(s)] = sum([v_val_Q_im[(s,m,i)] for m in range(Products) for i in range(Manufacturing_plants)])
        Summary_dict["LostSales_" + str(s)] = sum([v_val_U_km[(s,k,m)] for m in range(Products) for k in range(Market)])
        Summary_dict["OutsourceToDC_" + str(s)] = sum([v_val_T_ljm[(s,m,l,j)] for m in range(Products) for l in range(Outsourced) for j in range(Distribution)])
        Summary_dict["OutsourceToMarket_" + str(s)] = sum([v_val_T_lkm[(s,m,l,k)] for m in range(Products) for l in range(Outsourced) for k in range(Market)])        
        Summary_dict["alpha_vals_" + str(s)] = sum([v_val_alpha_kms[s,k,m] for k in range(Market) for m in range(Products)])
        
    for s in range(num_Scenarios):
        Cost_dict["InHouseShipping_" + str(s)] = get_shipping_costs(s,v_val_Y_ijm, v_val_Z_jkm, v_val_T_ljm, v_val_T_lkm)[0]
        Cost_dict["OutsourceShipping_" + str(s)] = get_shipping_costs(s,v_val_Y_ijm, v_val_Z_jkm, v_val_T_ljm, v_val_T_lkm)[1]
        Cost_dict["Production_" + str(s)] = get_production_cost(s,v_val_Q_im)
        Cost_dict["Purchasing_" + str(s)] = get_purchase_costs(s,v_val_V_lm, v_val_y_lm)
        Cost_dict["LostSales_" + str(s)] = get_lost_cost(s,v_val_U_km)    
    
    return

In [65]:
# Objective

def SetGrb_Obj():

    grb_expr = LinExpr()

    # Cost of opening
    OC_1 = 0
    OC_2 = 0
    for i in range(Manufacturing_plants):
        OC_1 += f_i[i]*x_i[i]
    for j in range(Distribution):
        OC_2 += f_j[j]*x_j[j]       
    
    total_shipment = 0    
    total_pr_cost = 0
    total_b_cost = 0
    total_l_cost = 0
    
    # Shipment 

    for s in range(num_Scenarios):
        ship_1 = 0
        ship_2 = 0
        ship_3 = 0
        ship_4 = 0
        for i in range(Manufacturing_plants):
            for j in range(Distribution):
                for m in range(Products):
                    ship_1 += Transportation_i_j[m][i][j]*Y_ijm[s,m,i,j]

        for j in range(Distribution):
            for k in range(Market):
                for m in range(Products):
                    ship_2 += Transportation_j_k[m][j][k]*Z_jkm[s,m,j,k]

        for l in range(Outsourced):
            for j in range(Distribution):
                for m in range(Products):
                    ship_3 += T_O_DC[m][l][j]*T_ljm[s,m,l,j]

        for l in range(Outsourced):
            for k in range(Market):
                for m in range(Products):
                    ship_4 += T_O_MZ[m][l][k]*T_lkm[s,m,l,k]
                    
        total_shipment += ship_1 + ship_2 + ship_3 + ship_4

        # Production
        pr_cost = 0
        for i in range(Manufacturing_plants):
            for m in range(Products):
                pr_cost += Manufacturing_costs[i][m]*Q_im[s,m,i]
                
        total_pr_cost += pr_cost

        # Buying from outsource cost
        b_cost = 0
        for l in range(Outsourced):
            for m in range(Products):
                b_cost += Supplier_cost[0][m][l]*V_lm[s,m,l]*(1 - y_lm[s,m,l]) + (Supplier_cost[0][m][l]*epsilon + Supplier_cost[1][m][l]*(V_lm[s,m,l] - epsilon))*y_lm[s,m,l]
                
        total_b_cost += b_cost

        #Lost Sales
        l_cost = 0
        for k in range(Market):
            for m in range(Products):
                l_cost += lost_sales[k][m]*U_km[s,k,m]
                
        total_l_cost += l_cost   

    # Penalties
    p_robust_penalties = 0
    for s in range(num_Scenarios):
        p_robust_penalties += penalty_weights[s]*delta_s[s]
        
    
    demand_penalties = 0
    for s in range(num_Scenarios):
        for k in range(Market):
            for m in range(Products): 
                demand_penalties += 100*alpha_kms[s,k,m]
        
    grb_expr += OC_1 + OC_2 + p_scen*(total_shipment + total_pr_cost + total_b_cost + total_l_cost + p_robust_penalties + demand_penalties)

    
    grbModel.setObjective(grb_expr, GRB.MINIMIZE)
    
    return 


In [66]:
# Model Constraints

def ModelCons():
    
    # Network Flow

    grbModel.addConstrs(Q_im[s,m,i] >= quicksum(Y_ijm[s,m,i,j] for j in range(Distribution)) 
                         for s in range(num_Scenarios) for i in range(Manufacturing_plants) for m in range(Products))

    grbModel.addConstrs((quicksum(Y_ijm[s,m,i,j] for i in range(Manufacturing_plants)) +
                         quicksum(T_ljm[s,m,l,j] for l in range(Outsourced))) >= quicksum(Z_jkm[s,m,j,k] for k in range(Market))                        
                        for s in range(num_Scenarios) for j in range(Distribution) for m in range(Products))
    
    
    grbModel.addConstrs(quicksum(Z_jkm[s,m,j,k] for j in range(Distribution)) + 
                        quicksum(T_lkm[s,m,l,k] for l in range(Outsourced)) + 
                        alpha_kms[s,k,m] + U_km[s,k,m] >= demand[s][m][k] for s in range(num_Scenarios) for m in range(Products)
                        for k in range(Market))
        
                    
    # Purchasing Constraints (everything purchased from outsourced facilities must be shipped)
    grbModel.addConstrs(V_lm[s,m,l] >= quicksum(T_ljm[s,m,l,j] for j in range(Distribution)) + 
                        quicksum(T_lkm[s,m,l,k] for k in range(Market)) for s in range(num_Scenarios) 
                        for m in range(Products) for l in range(Outsourced))
    
    
    # Capacity Constraints
    grbModel.addConstrs(quicksum(volume[m]*Q_im[s,m,i] for m in range(Products)) <= Scenarios[s][0][i]*Capacities_i[i]*x_i[i] 
                        for s in range(num_Scenarios) for i in range(Manufacturing_plants))
    
    grbModel.addConstrs(quicksum(volume[m]*Y_ijm[s,m,i,j] for i in range(Manufacturing_plants) for m in range(Products)) +
                        quicksum(volume[m]*T_ljm[s,m,l,j] for l in range(Outsourced) for m in range(Products)) <= 
                        Scenarios[s][1][j]*Capacities_j[j]*x_j[j] for s in range(num_Scenarios) for s in range(num_Scenarios)
                        for j in range(Distribution))
    
    grbModel.addConstrs((V_lm[s,m,l] <= (Capacities_l[m][l])) for s in range(num_Scenarios)
                        for l in range(Outsourced) for m in range(Products))
    
    
    # Indicator variable constraints for step function (25 is arbitrary)
    grbModel.addConstrs(V_lm[s,m,l] >= (epsilon + 1)*y_lm[s,m,l] for s in range(num_Scenarios) 
                        for m in range(Products) for l in range(Outsourced))
    
    grbModel.addConstrs((V_lm[s,m,l] - (Capacities_l[m][l] - epsilon)*y_lm[s,m,l])
                         <= epsilon for s in range(num_Scenarios) for m in range(Products) for l in range(Outsourced))   
    
    
    # Resilience Metric (demand recovery must be above 80%)
    grbModel.addConstrs(quicksum(Z_jkm[s,m,j,k] for j in range(Distribution)) + 
                        quicksum(T_lkm[s,m,l,k] for l in range(Outsourced)) + 
                        alpha_kms[s,k,m] >= dem_recovery*demand[s][m][k] for s in range(num_Scenarios) for m in range(Products)
                        for k in range(Market))   

    
    # P-robust metric    
    grbModel.addConstrs(quicksum(f_i[i]*x_i[i] for i in range(Manufacturing_plants))
                        + quicksum(f_j[j]*x_j[j] for j in range(Distribution))
                        + quicksum(Transportation_i_j[m][i][j]*Y_ijm[s,m,i,j] for i in range(Manufacturing_plants) 
                                  for j in range(Distribution) for m in range(Products))
                        + quicksum(Transportation_j_k[m][j][k]*Z_jkm[s,m,j,k] for j in range(Distribution)
                                   for k in range(Market) for m in range(Products))
                        + quicksum(T_O_DC[m][l][j]*T_ljm[s,m,l,j] for l in range(Outsourced) for j in range(Distribution)
                                   for m in range(Products))
                        + quicksum(T_O_MZ[m][l][k]*T_lkm[s,m,l,k] for l in range(Outsourced) for k in range(Market)
                                   for m in range(Products))
                        + quicksum(Manufacturing_costs[i][m]*Q_im[s,m,i] for i in range(Manufacturing_plants) 
                                   for m in range(Products))
                        + quicksum(Supplier_cost[0][m][l]*V_lm[s,m,l] + Supplier_cost[1][m][l]*y_lm[s,m,l]
                                   for l in range(Outsourced) for m in range(Products))
                        + quicksum(lost_sales[k][m]*U_km[s,k,m] for k in range(Market) for m in range(Products)) - delta_s[s]
                        
                        <= (1 + p)*scen_objs[s] for s in range(num_Scenarios))                                 
                                                                    
    
    return   


In [67]:
def get_opening_costs(x1, x2):
    
    # Cost of opening
    OC_1 = 0
    OC_2 = 0
    for i in range(Manufacturing_plants):
        OC_1 += f_i[i]*x1[i]
    for j in range(Distribution):
        OC_2 += f_j[j]*x2[j]

    Opening = np.round(OC_1 + OC_2)
    
    return(Opening)
   
def get_shipping_costs(scen, Y, Z, T1, T2):
    ship_1 = 0
    ship_2 = 0
    ship_3 = 0
    ship_4 = 0

    # Shipment
    for i in range(Manufacturing_plants):
        for j in range(Distribution):
            for m in range(Products):
                ship_1 += Transportation_i_j[m][i][j]*Y[scen, m,i,j]

    for j in range(Distribution):
        for k in range(Market):
            for m in range(Products):
                ship_2 += Transportation_j_k[m][j][k]*Z[scen,m,j,k]

    for l in range(Outsourced):
        for j in range(Distribution):
            for m in range(Products):
                ship_3 += T_O_DC[m][l][j]*T1[scen,m,l,j]

    for l in range(Outsourced):
        for k in range(Market):
            for m in range(Products):
                ship_4 += T_O_MZ[m][l][k]*T2[scen,m,l,k]
    
    in_house_shipping = np.round(ship_1 + ship_2)

    outsourced_shipping = np.round(ship_3 + ship_4)
    
    return(in_house_shipping, outsourced_shipping)

def get_production_cost(scen, Q):

    # Production
    pr_cost = 0
    for i in range(Manufacturing_plants):
        for m in range(Products):
            pr_cost += Manufacturing_costs[i][m]*Q[scen,m,i]
            
    return(np.round(pr_cost))

def get_purchase_costs(scen, V, y):    

    # Buying from outsource cost
    b_cost = 0
    for l in range(Outsourced):
        for m in range(Products):
            b_cost += Supplier_cost[0][m][l]*V[scen,m,l]*(1 - y[scen,m,l]) + (Supplier_cost[0][m][l]*epsilon + Supplier_cost[1][m][l]*(V[scen,m,l] - epsilon))*y[scen,m,l]          

    return(np.round(b_cost))

def get_lost_cost(scen,U):
    
    #Lost Sales
    l_cost = 0
    for k in range(Market):
        for m in range(Products):
            l_cost += lost_sales[k][m]*U[scen,k,m]
            
    return(np.round(l_cost))

def get_r_penalty_costs(delta):
    penalties = 0
    for s in range(num_Scenarios):
        penalties += penalty_weights[s]*delta[s]
    return(penalties)

def get_d_penalty_costs(alpha):
    penalties = 0
    for s in range(num_Scenarios):
        for k in range(Market):
            for m in range(Products): 
                penalties += 100*alpha[s,k,m]
    return(penalties)

In [68]:
def run_Model():    
        
    SetGurobiModel()
    SolveModel()

In [69]:
def dict_to_dataframe(Dict):
    return pd.DataFrame([list(Dict.values())], columns = list(Dict.keys()))

In [70]:
run_Model()

Parameter OutputFlag unchanged
   Value: 1  Min: 0  Max: 1  Default: 1
Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (win64)
Optimize a model with 2268 rows, 1958 columns and 17962 nonzeros
Model fingerprint: 0x3783e6cc
Model has 84 quadratic objective terms
Variable types: 189 continuous, 1769 integer (89 binary)
Coefficient statistics:
  Matrix range     [4e-02, 1e+03]
  Objective range  [2e-03, 2e+02]
  QObjective range [3e-01, 6e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [8e-01, 4e+03]
Found heuristic solution: objective 1.601397e+12
Presolve removed 1513 rows and 587 columns
Presolve time: 0.02s
Presolved: 839 rows, 1455 columns, 5046 nonzeros
Found heuristic solution: objective 18742.857143
Variable types: 147 continuous, 1308 integer (112 binary)

Root relaxation: objective 1.573969e+03, 604 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Tim

In [71]:
num_correct = 0
for s in range(num_Scenarios):
    if (Summary_dict['Purchasing_'+str(s)] + Summary_dict['Production_' + str(s) ] + Summary_dict['LostSales_' + str(s)] + Summary_dict['alpha_vals_' +str(s)]) == np.sum(demand[s]):
        num_correct += 1
num_correct == num_Scenarios

True

In [72]:
Purchasing = [Summary_dict['Purchasing_' + str(s)] for s in range(num_Scenarios)]
Production = [Summary_dict['Production_' + str(s)] for s in range(num_Scenarios)]
LostSales = [Summary_dict['LostSales_' + str(s)] for s in range(num_Scenarios)]
OutsourceToDC = [Summary_dict['OutsourceToDC_' + str(s)] for s in range(num_Scenarios)]
OutsourceToMarket = [Summary_dict['OutsourceToMarket_' + str(s)] for s in range(num_Scenarios)]

In [73]:
Unit_df = pd.DataFrame(list(zip(Purchasing, Production, LostSales, OutsourceToDC, OutsourceToMarket)), 
             columns = ["Purchasing", "Production", "LostSales", "OutsourceToDC", "OutsourceToMarket"])

In [74]:
Unit_df

,Purchasing,Production,LostSales,OutsourceToDC,OutsourceToMarket
0,0.0,164.0,0.0,0.0,0.0
1,0.0,147.0,0.0,0.0,0.0
2,0.0,249.0,0.0,0.0,0.0
3,0.0,133.0,0.0,0.0,0.0
4,0.0,193.0,0.0,0.0,0.0
5,0.0,168.0,0.0,0.0,0.0
6,64.0,103.0,0.0,0.0,64.0
7,0.0,202.0,0.0,0.0,0.0
8,0.0,197.0,0.0,0.0,0.0
9,0.0,194.0,0.0,0.0,0.0


In [75]:
Purchasing_cost = [Cost_dict['Purchasing_' + str(s)] for s in range(num_Scenarios)]
Production_cost = [Cost_dict['Production_' + str(s)] for s in range(num_Scenarios)]
LostSales_cost = [Cost_dict['LostSales_' + str(s)] for s in range(num_Scenarios)]
InHouseShipping = [Cost_dict['InHouseShipping_' + str(s)] for s in range(num_Scenarios)]
OutsourceShipping = [Cost_dict['OutsourceShipping_' + str(s)] for s in range(num_Scenarios)]

In [76]:
Cost_dict['Opening'] + p_scen*(sum(Purchasing_cost) + sum(Production_cost) + sum(InHouseShipping) + sum(OutsourceShipping) + sum(LostSales_cost) + Cost_dict['r_Penalties'] + Cost_dict['d_Penalties'])

1763.1133888542563

In [77]:
grbModel.objVal

1763.300635193303

In [78]:
Cost_df = pd.DataFrame(list(zip(Purchasing_cost, Production_cost, LostSales_cost, InHouseShipping, OutsourceShipping)), 
             columns = ["Purchasing", "Production", "LostSales", "InHouseShipping", "OutsourceShipping"])

In [79]:
Cost_df

,Purchasing,Production,LostSales,InHouseShipping,OutsourceShipping
0,0.0,182.0,0.0,318.0,0.0
1,0.0,161.0,0.0,304.0,0.0
2,0.0,291.0,0.0,462.0,0.0
3,0.0,145.0,0.0,215.0,0.0
4,0.0,220.0,0.0,424.0,0.0
5,0.0,188.0,0.0,456.0,0.0
6,820.0,113.0,0.0,251.0,373.0
7,0.0,256.0,0.0,339.0,0.0
8,0.0,258.0,0.0,386.0,0.0
9,0.0,247.0,0.0,386.0,0.0


In [80]:
Summary_dict["OpenMPs"]

2.0

In [81]:
Summary_dict["OpenDCs"]

3.0

In [82]:
scen_costs = [Cost_dict['Opening'] + Purchasing_cost[s] + Production_cost[s] + InHouseShipping[s] + OutsourceShipping[s] + LostSales_cost[s] for s in range(num_Scenarios)]

In [83]:
(np.array(scen_costs) - np.array(scen_objs))/np.array(scen_objs)

array([0.15832868, 0.19469758, 0.04720078, 0.24211591, 0.07147987,
       0.12770589, 0.13856671, 0.15956478, 0.16202997, 0.21657974,
       0.30245801, 0.11316074, 0.14806386, 0.10401686, 0.10775532,
       0.24377936, 0.14206052, 0.15005567, 0.1904899 , 0.15833508,
       0.15790834])

In [84]:
Cost_dict['r_Penalties']

173.38116593938213

In [85]:
Cost_dict['d_Penalties']

0.0

## Comparing different weights for objective values 